In [1]:
import numpy as np
import pandas as pd

In [23]:
#For text preprocessing
import re
import spacy
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
import unidecode
import wordninja

In [3]:
# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
# Importing Gensim
import gensim
from gensim import corpora

In [5]:
df=pd.read_csv('Assignment 2_Web Data.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SNO                 10000 non-null  int64 
 1   Date                10000 non-null  object
 2   Time                10000 non-null  object
 3   Media Type          10000 non-null  object
 4   Site Name           10000 non-null  object
 5   Site Domain         10000 non-null  object
 6   Mention URL         8308 non-null   object
 7   Publisher Name      9257 non-null   object
 8   Publisher Username  7752 non-null   object
 9   title               9980 non-null   object
 10  Mention Content     9996 non-null   object
 11  Country             10000 non-null  object
 12  State               5389 non-null   object
 13  City                3478 non-null   object
 14  Language            10000 non-null  object
dtypes: int64(1), object(14)
memory usage: 1.1+ MB


In [7]:
df.head()

,SNO,Date,Time,Media Type,Site Name,Site Domain,Mention URL,Publisher Name,Publisher Username,title,Mention Content,Country,State,City,Language
0,1,2021-03-15,23:41:00 +0530 IST,Social network,Facebook,http://www.facebook.com,NaN,Anonymous user,NaN,Motorola,90Hz #motog10power #motog30 #AsliAllRounderCha...,India,NaN,NaN,English
1,2,2021-03-15,17:27:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/RajibGogoi668/status/137143...,Rajib Gogoi,RajibGogoi668,RT @ManojSaru: Another Giveaway for #TechGyan ...,RT @ManojSaru: Another Giveaway for #TechGyan ...,India,Assam,Jorhat,English
2,3,2021-03-15,21:03:00 +0530 IST,Social network,Facebook,http://www.facebook.com,NaN,Anonymous user,NaN,Motorola,Wow Magnificent With Amazing Tech Features #Po...,India,NaN,NaN,English
3,4,2021-03-15,16:58:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/VaishaliMaisur2/status/1371...,Vaishali Maisuria,VaishaliMaisur2,@motorolaindia A3) Think Shield Follow all pla...,@motorolaindia A3) Think Shield Follow all pla...,India,Gujarat,Surat,English
4,5,2021-03-15,22:15:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/dhanve_pranay/status/137150...,Panu Dhanve,dhanve_pranay,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,India,Maharashtra,Wangi,English


In [8]:
# Checking the Language column to see how maany languages are present
df.Language.value_counts()

English          9153
Not available     691
Hindi              99
Tamil              41
Telugu              6
Marathi             3
Gujarati            3
Panjabi             2
Bengali             2
Name: Language, dtype: int64

#### Since only 140 out 10000 rows are other languages. So I am dropping away the other languages and selecting only English as this will not effect the Topic that much and including it will add a little more work to do. So, considering the time and importance taking only the English language in consideration.

In [9]:
#Selecting only entity with English Language.
df_English=df[df.Language=='English']

In [10]:
df_English.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9153 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SNO                 9153 non-null   int64 
 1   Date                9153 non-null   object
 2   Time                9153 non-null   object
 3   Media Type          9153 non-null   object
 4   Site Name           9153 non-null   object
 5   Site Domain         9153 non-null   object
 6   Mention URL         7462 non-null   object
 7   Publisher Name      8481 non-null   object
 8   Publisher Username  6975 non-null   object
 9   title               9133 non-null   object
 10  Mention Content     9149 non-null   object
 11  Country             9153 non-null   object
 12  State               4630 non-null   object
 13  City                2737 non-null   object
 14  Language            9153 non-null   object
dtypes: int64(1), object(14)
memory usage: 1.1+ MB


In [11]:
df_English['Mention Content'].isnull().sum()

4

In [12]:
#selecting only SNO and Mention Content column as we have to Topic modeeling for comments only.
df_Eng_doc=df_English.loc[:,['SNO','Mention Content']]

In [13]:
df_Eng_doc.head()

,SNO,Mention Content
0,1,90Hz #motog10power #motog30 #AsliAllRounderCha...
1,2,RT @ManojSaru: Another Giveaway for #TechGyan ...
2,3,Wow Magnificent With Amazing Tech Features #Po...
3,4,@motorolaindia A3) Think Shield Follow all pla...
4,5,RT @OnePlusClub: #OnePlus9Pro in Morning Mist


In [14]:
df_Eng_doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9153 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SNO              9153 non-null   int64 
 1   Mention Content  9149 non-null   object
dtypes: int64(1), object(1)
memory usage: 214.5+ KB


In [15]:
#Dropping the rows which do not contain any Data
df_Eng_doc.dropna(inplace=True)

In [16]:
df_Eng_doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9149 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SNO              9149 non-null   int64 
 1   Mention Content  9149 non-null   object
dtypes: int64(1), object(1)
memory usage: 214.4+ KB


In [17]:
#Looking into the Second comment to see the extent 
df_Eng_doc['Mention Content'][1]

"RT @ManojSaru: Another Giveaway for #TechGyan Family🎁🎁 Yes i am giving Away this OPPO's New Smartphone #OPPOF19Pro So Don't Forget to Participate Participate Link : https://www.youtube.com/watch?v=_QhoNN75iPk&feature=youtu.be Feel Free to LIKE & Retweet This Post #Giveaway #FlauntYourNights"

In [24]:
# stop loss words 
stop = set(stopwords.words('english'))

# punctuation 
exclude = set(string.punctuation) 

# lemmatization
lemma = WordNetLemmatizer() 

def clean_doc(text):
    
    # lower-case all characters
    text=text.lower()
    
    # remove twitter handles
    text= re.sub(r'@\S+', '',text) 
    
    # remove urls
    text= re.sub(r'http\S+', '',text) 
    text= re.sub(r'pic.\S+', '',text)
      
    # replace unidecode characters
    text=unidecode.unidecode(text)
      
    # regex only keeps characters
    text= re.sub(r"[^a-zA-Z+']", ' ',text)
    
    # keep words with length>1 only
    text=re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ') 

    # split words like 'whatisthis' to 'what is this'
    def preprocess_wordninja(sentence):      
        def split_words(x):
            x=wordninja.split(x)
            x= [word for word in x if len(word)>1]
            return x
        new_sentence=[ ' '.join(split_words(word)) for word in sentence.split() ]
        return ' '.join(new_sentence)
    
    text=preprocess_wordninja(text)
    
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in text.lower().split() if i not in stop])
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    
    # remove punctuations + normalize the text
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
 
    # regex removes repeated spaces, strip removes leading and trailing spaces
    text= re.sub("\s[\s]+", " ",normalized).strip()  
    
    return text

In [25]:
# clean data stored in a new list
clean_corpus = [clean_doc(doc).split() for doc in df_Eng_doc['Mention Content']]

In [26]:
for i in clean_corpus[:2]:
    print(len(i))

7
27


In [27]:
clean_corpus[:2]

[['hz', 'moto', 'power', 'moto', 'li', 'rounder', 'challenge'],
 ['rt',
  'another',
  'giveaway',
  'tech',
  'gy',
  'family',
  'yes',
  'giving',
  'away',
  'opp',
  'o',
  'new',
  'smartphone',
  'opp',
  'pro',
  'forget',
  'participate',
  'participate',
  'link',
  'feel',
  'free',
  'like',
  'tweet',
  'post',
  'giveaway',
  'flaunt',
  'night']]

In [28]:
# Creating the term dictionary of our courpus that is of all the words (Sepcific to Genism syntax perspective), 
# where every unique term is assigned an index. 

dict_ = corpora.Dictionary(clean_corpus)

print(dict_)

Dictionary(7083 unique tokens: ['challenge', 'hz', 'li', 'moto', 'power']...)


#### Now, the next step is to convert the corpus (the list of documents) into a document-term Matrix using the dictionary that we had prepared above. (The vectorizer used here is the Bag of Words).

In [29]:
# Converting list of documents (corpus) into Document Term Matrix using the dictionary 
doc_term_matrix = [dict_.doc2bow(i) for i in clean_corpus]
doc_term_matrix[:2]

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1)],
 [(6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 2),
  (22, 2),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1)]]

The output implies:

Document wise we have the index of the word and its frequency.
The 0th word is repeated 1 time, then the 1st word repeated 1 and so on ...

## 4. Implementation of LDA

In [30]:
# Creating the object for LDA model using gensim library

Lda = gensim.models.ldamodel.LdaModel

In [31]:
# Running and Training LDA model on the document term matrix.

ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dict_, passes=1, random_state=0, eval_every=None)

In [32]:
ldamodel.print_topics()

[(0,
  '0.023*"phone" + 0.020*"iphone" + 0.020*"business" + 0.020*"ki" + 0.017*"real" + 0.016*"nah" + 0.016*"thanks" + 0.016*"ye" + 0.015*"market" + 0.015*"hai"'),
 (1,
  '0.038*"watch" + 0.034*"price" + 0.022*"mane" + 0.015*"le" + 0.015*"moto" + 0.012*"htc" + 0.012*"rounder" + 0.011*"wildfire" + 0.011*"motorola" + 0.010*"r"'),
 (2,
  '0.290*"trak" + 0.157*"tech" + 0.145*"keep" + 0.007*"sharma" + 0.007*"ghosh" + 0.007*"moto" + 0.007*"rounder" + 0.006*"management" + 0.006*"supa" + 0.006*"power"'),
 (3,
  '0.046*"alert" + 0.037*"rt" + 0.037*"contest" + 0.034*"opp" + 0.029*"pro" + 0.021*"chance" + 0.020*"stand" + 0.019*"mall" + 0.016*"series" + 0.016*"camera"'),
 (4,
  '0.060*"samsung" + 0.048*"galaxy" + 0.047*"review" + 0.030*"giveaway" + 0.030*"sensor" + 0.027*"pro" + 0.022*"even" + 0.022*"opp" + 0.022*"get" + 0.021*"note"'),
 (5,
  '0.127*"moto" + 0.110*"rounder" + 0.085*"li" + 0.072*"power" + 0.067*"challenge" + 0.035*"motorola" + 0.022*"join" + 0.016*"mah" + 0.015*"shield" + 0.013*"s

### 4a. Extracting Topics from the Corpus 

In [33]:
print(ldamodel.print_topics(num_topics=6, num_words=5))

# num_topics mean: how many topics want to extract 
# num_words: the number of words that want per topic

[(8, '0.062*"sensor" + 0.040*"mode" + 0.037*"night" + 0.033*"contest" + 0.031*"li"'), (2, '0.290*"trak" + 0.157*"tech" + 0.145*"keep" + 0.007*"sharma" + 0.007*"ghosh"'), (15, '0.031*"rounder" + 0.030*"moto" + 0.025*"li" + 0.023*"motorola" + 0.017*"challenge"'), (1, '0.038*"watch" + 0.034*"price" + 0.022*"mane" + 0.015*"le" + 0.015*"moto"'), (16, '0.056*"rounder" + 0.035*"moto" + 0.033*"rt" + 0.032*"li" + 0.031*"challenge"'), (12, '0.069*"mi" + 0.042*"red" + 0.031*"note" + 0.029*"phone" + 0.025*"good"')]


### 4b. Assigning the topics to the documents

Pass the doc_term_matrix to the LDA model:

In [34]:
# printing the topic associations with the documents
count = 0
for i in ldamodel[doc_term_matrix]:
    print("doc : ",count,i)
    count += 1
    if count>5:
        break

doc :  0 [(5, 0.8812497)]
doc :  1 [(11, 0.9660713)]
doc :  2 [(2, 0.09913396), (5, 0.7174303), (6, 0.10614443)]
doc :  3 [(5, 0.8326405), (7, 0.092356324)]
doc :  4 [(9, 0.8642854)]
doc :  5 [(10, 0.9176556), (16, 0.059266157)]


### Getting Dominant_Topic for each Entity 

In [35]:
count = 0
sent_topics_df = pd.DataFrame()
for i, row in enumerate(ldamodel[doc_term_matrix]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break

    # Add original text to the end of the output

In [36]:
sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
contents = pd.Series(clean_corpus)
sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)

In [37]:
sent_topics_df.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,5.0,0.8812,"moto, rounder, li, power, challenge, motorola,...","[hz, moto, power, moto, li, rounder, challenge]"
1,11.0,0.9661,"opp, giveaway, participate, like, pro, tech, o...","[rt, another, giveaway, tech, gy, family, yes,..."
2,5.0,0.7173,"moto, rounder, li, power, challenge, motorola,...","[wow, magnificent, amazing, tech, feature, pow..."
3,5.0,0.8324,"moto, rounder, li, power, challenge, motorola,...","[think, shield, follow, platform, li, rounder,..."
4,9.0,0.8643,"one, plus, pro, mist, rt, morning, march, new,...","[rt, one, plus, pro, morning, mist]"


In [38]:
sent_topics_df.shape

(9149, 4)

In [39]:
# Format
df_dominant_topic = sent_topics_df.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [40]:
#show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,5.0,0.8812,"moto, rounder, li, power, challenge, motorola,...","[hz, moto, power, moto, li, rounder, challenge]"
1,1,11.0,0.9661,"opp, giveaway, participate, like, pro, tech, o...","[rt, another, giveaway, tech, gy, family, yes,..."
2,2,5.0,0.7173,"moto, rounder, li, power, challenge, motorola,...","[wow, magnificent, amazing, tech, feature, pow..."
3,3,5.0,0.8324,"moto, rounder, li, power, challenge, motorola,...","[think, shield, follow, platform, li, rounder,..."
4,4,9.0,0.8643,"one, plus, pro, mist, rt, morning, march, new,...","[rt, one, plus, pro, morning, mist]"
5,5,10.0,0.9177,"series, pro, real, opp, rt, mp, phone, oled, u...","[rt, contest, alert, tag, u, post, image, one,..."
6,6,5.0,0.8944,"moto, rounder, li, power, challenge, motorola,...","[mah, battery, li, rounder, challenge, moto, p..."
7,7,13.0,0.6833,"galaxy, rt, heart, bts, winner, display, camer...","[design, battery]"
8,8,11.0,0.9661,"opp, giveaway, participate, like, pro, tech, o...","[rt, another, giveaway, tech, gy, family, yes,..."
9,9,2.0,0.8100,"trak, tech, keep, sharma, ghosh, moto, rounder...","[trak, tech, keep, trak]"


### Combine the result with our original Dataset

In [41]:
df_Eng_doc.head()

,SNO,Mention Content
0,1,90Hz #motog10power #motog30 #AsliAllRounderCha...
1,2,RT @ManojSaru: Another Giveaway for #TechGyan ...
2,3,Wow Magnificent With Amazing Tech Features #Po...
3,4,@motorolaindia A3) Think Shield Follow all pla...
4,5,RT @OnePlusClub: #OnePlus9Pro in Morning Mist


In [42]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,5.0,0.8812,"moto, rounder, li, power, challenge, motorola,...","[hz, moto, power, moto, li, rounder, challenge]"
1,1,11.0,0.9661,"opp, giveaway, participate, like, pro, tech, o...","[rt, another, giveaway, tech, gy, family, yes,..."
2,2,5.0,0.7173,"moto, rounder, li, power, challenge, motorola,...","[wow, magnificent, amazing, tech, feature, pow..."
3,3,5.0,0.8324,"moto, rounder, li, power, challenge, motorola,...","[think, shield, follow, platform, li, rounder,..."
4,4,9.0,0.8643,"one, plus, pro, mist, rt, morning, march, new,...","[rt, one, plus, pro, morning, mist]"


In [43]:
df_a=df_dominant_topic.iloc[:,:3]

In [44]:
df_Eng_doc_segmented=pd.concat([df_Eng_doc,df_dominant_topic],axis=1)

In [45]:
df_Eng_doc_segmented.head()

,SNO,Mention Content,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,1.0,90Hz #motog10power #motog30 #AsliAllRounderCha...,0.0,5.0,0.8812,"moto, rounder, li, power, challenge, motorola,...","[hz, moto, power, moto, li, rounder, challenge]"
1,2.0,RT @ManojSaru: Another Giveaway for #TechGyan ...,1.0,11.0,0.9661,"opp, giveaway, participate, like, pro, tech, o...","[rt, another, giveaway, tech, gy, family, yes,..."
2,3.0,Wow Magnificent With Amazing Tech Features #Po...,2.0,5.0,0.7173,"moto, rounder, li, power, challenge, motorola,...","[wow, magnificent, amazing, tech, feature, pow..."
3,4.0,@motorolaindia A3) Think Shield Follow all pla...,3.0,5.0,0.8324,"moto, rounder, li, power, challenge, motorola,...","[think, shield, follow, platform, li, rounder,..."
4,5.0,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,4.0,9.0,0.8643,"one, plus, pro, mist, rt, morning, march, new,...","[rt, one, plus, pro, morning, mist]"


In [46]:
df_final=pd.merge(df,df_Eng_doc_segmented,how='left',on='SNO')

In [47]:
df_final.head()

,SNO,Date,Time,Media Type,Site Name,Site Domain,Mention URL,Publisher Name,Publisher Username,title,...,Country,State,City,Language,Mention Content_y,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,1,2021-03-15,23:41:00 +0530 IST,Social network,Facebook,http://www.facebook.com,NaN,Anonymous user,NaN,Motorola,...,India,NaN,NaN,English,90Hz #motog10power #motog30 #AsliAllRounderCha...,0.0,5.0,0.8812,"moto, rounder, li, power, challenge, motorola,...","[hz, moto, power, moto, li, rounder, challenge]"
1,2,2021-03-15,17:27:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/RajibGogoi668/status/137143...,Rajib Gogoi,RajibGogoi668,RT @ManojSaru: Another Giveaway for #TechGyan ...,...,India,Assam,Jorhat,English,RT @ManojSaru: Another Giveaway for #TechGyan ...,1.0,11.0,0.9661,"opp, giveaway, participate, like, pro, tech, o...","[rt, another, giveaway, tech, gy, family, yes,..."
2,3,2021-03-15,21:03:00 +0530 IST,Social network,Facebook,http://www.facebook.com,NaN,Anonymous user,NaN,Motorola,...,India,NaN,NaN,English,Wow Magnificent With Amazing Tech Features #Po...,2.0,5.0,0.7173,"moto, rounder, li, power, challenge, motorola,...","[wow, magnificent, amazing, tech, feature, pow..."
3,4,2021-03-15,16:58:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/VaishaliMaisur2/status/1371...,Vaishali Maisuria,VaishaliMaisur2,@motorolaindia A3) Think Shield Follow all pla...,...,India,Gujarat,Surat,English,@motorolaindia A3) Think Shield Follow all pla...,3.0,5.0,0.8324,"moto, rounder, li, power, challenge, motorola,...","[think, shield, follow, platform, li, rounder,..."
4,5,2021-03-15,22:15:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/dhanve_pranay/status/137150...,Panu Dhanve,dhanve_pranay,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,...,India,Maharashtra,Wangi,English,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,4.0,9.0,0.8643,"one, plus, pro, mist, rt, morning, march, new,...","[rt, one, plus, pro, morning, mist]"


In [48]:
df_Seg=df_final.loc[:,['SNO','Dominant_Topic']]

In [49]:
df_Seg.head(20)

,SNO,Dominant_Topic
0,1,5.0
1,2,11.0
2,3,5.0
3,4,5.0
4,5,9.0
5,6,10.0
6,7,5.0
7,8,NaN
8,9,11.0
9,10,2.0
